<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/JARVIS_LAMMPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# %env LD_PRELOAD=
# !apt install cmake build-essential git ccache
# Clone the LAMMPS git
if not os.path.exists('lammps'):
   os.system('git clone https://github.com/lammps/lammps.git')


In [2]:
os.chdir('lammps/src')


In [3]:
!make yes-MANYBODY
!make serial

Installing package MANYBODY
make[1]: Entering directory '/content/lammps/src/STUBS'
g++ -O -fPIC -I.  -c mpi.cpp
ar rs libmpi_stubs.a mpi.o
ar: creating libmpi_stubs.a
make[1]: Leaving directory '/content/lammps/src/STUBS'
Gathering installed package information (may take a little while)
make[1]: Entering directory '/content/lammps/src'
Gathering git version information
make[1]: Leaving directory '/content/lammps/src'
Compiling LAMMPS for machine serial
make[1]: Entering directory '/content/lammps/src/Obj_serial'
cc -O -o fastdep.exe ../DEPEND/fastdep.c
make[1]: Leaving directory '/content/lammps/src/Obj_serial'
make[1]: Entering directory '/content/lammps/src/Obj_serial'
g++ -g -O3   -DLAMMPS_GZIP -DLAMMPS_MEMALIGN=64    -I../STUBS     -c ../main.cpp
g++ -g -O3   -DLAMMPS_GZIP -DLAMMPS_MEMALIGN=64    -I../STUBS     -c ../angle.cpp
g++ -g -O3   -DLAMMPS_GZIP -DLAMMPS_MEMALIGN=64    -I../STUBS     -c ../angle_deprecated.cpp
g++ -g -O3   -DLAMMPS_GZIP -DLAMMPS_MEMALIGN=64    -I../STUBS  

In [4]:
!export lammps_cmd="/content/lammps/src/lmp_serial"

In [5]:
!wget https://www.ctcms.nist.gov/~knc6/DOWNLOADS/Al_zhou.eam.alloy
!wget https://raw.githubusercontent.com/usnistgov/jarvis/master/jarvis/tasks/lammps/templates/inelast.mod

--2025-01-19 17:51:59--  https://www.ctcms.nist.gov/~knc6/DOWNLOADS/Al_zhou.eam.alloy
Resolving www.ctcms.nist.gov (www.ctcms.nist.gov)... 129.6.13.19, 2610:20:6005:13::19
Connecting to www.ctcms.nist.gov (www.ctcms.nist.gov)|129.6.13.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 710486 (694K) [application/zip]
Saving to: ‘Al_zhou.eam.alloy’

Al_zhou.eam.alloy   100%[===================>] 693.83K  4.12MB/s    in 0.2s    

2025-01-19 17:52:00 (4.12 MB/s) - ‘Al_zhou.eam.alloy’ saved [710486/710486]

--2025-01-19 17:52:00--  https://raw.githubusercontent.com/usnistgov/jarvis/master/jarvis/tasks/lammps/templates/inelast.mod
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6416 (6.3K) [text/plain]
Saving to: ‘inelast.mod’

inelast.mo

In [6]:
!pip install -q jarvis-tools spglib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.8 MB/s eta 0:00:00


In [7]:
from jarvis.tasks.lammps.lammps import LammpsJob, JobFactory
from jarvis.core.atoms import Atoms
from jarvis.db.figshare import get_jid_data
from jarvis.analysis.structure.spacegroup import Spacegroup3D
tmp_dict = get_jid_data(jid="JVASP-816", dataset="dft_3d")["atoms"]
atoms = Atoms.from_dict(tmp_dict)
# Get conventional cell
spg = Spacegroup3D(atoms)
cvn_atoms = spg.conventional_standard_structure
ff = "/content/lammps/src/Al_zhou.eam.alloy"
mod = "/content/lammps/src/inelast.mod"
cmd = "/content/lammps/src/lmp_serial<in.main>out"

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699


100%|██████████| 40.8M/40.8M [00:02<00:00, 18.2MiB/s]


Loading the zipfile...
Loading completed.


/usr/local/lib/python3.11/dist-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['number']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['international']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


In [8]:
parameters = {
    "pair_style": "eam/alloy",
    "pair_coeff": ff,
    "atom_style": "charge",
    "control_file": mod,
}
# Test LammpsJob
lmp = LammpsJob(atoms=cvn_atoms, parameters=parameters, lammps_cmd=cmd, jobname="Test").runjob()



In [9]:
!ls

accelerator_kokkos.h		  fix_efield.h			      ntopo_angle_partial.h
accelerator_omp.h		  fix_enforce2d.cpp		      ntopo_angle_template.cpp
ADIOS				  fix_enforce2d.h		      ntopo_angle_template.h
Al_zhou.eam.alloy		  fix_evaporate.cpp		      ntopo_bond_all.cpp
AMOEBA				  fix_evaporate.h		      ntopo_bond_all.h
angle.cpp			  fix_external.cpp		      ntopo_bond_partial.cpp
angle_deprecated.cpp		  fix_external.h		      ntopo_bond_partial.h
angle_deprecated.h		  fix_gravity.cpp		      ntopo_bond_template.cpp
angle.h				  fix_gravity.h			      ntopo_bond_template.h
angle_hybrid.cpp		  fix_group.cpp			      ntopo.cpp
angle_hybrid.h			  fix_group.h			      ntopo_dihedral_all.cpp
angle_write.cpp			  fix.h				      ntopo_dihedral_all.h
angle_write.h			  fix_halt.cpp			      ntopo_dihedral_partial.cpp
angle_zero.cpp			  fix_halt.h			      ntopo_dihedral_partial.h
angle_zero.h			  fix_heat.cpp			      ntopo_dihedral_template.cpp
arg_info.cpp			  fix_heat.h			      ntopo_dihedral_template.h
arg_inf

In [10]:
!pip install -q phonopy ase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 592.9/592.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.0 MB/s eta 0:00:00


In [11]:
# Test high-throughput
job_fact = JobFactory(pair_style="eam/alloy", name="my_first_lammps_run")
job_fact.all_props_eam_alloy(atoms=cvn_atoms, ff_path=ff, lammps_cmd=cmd)

en, final_str, forces -3.58 System
1.0
4.081654900784626 0.0 0.0
0.0 4.081654900784625 0.0
0.0 0.0 4.081654900784626
Al 
4 
direct
-0.0017449929298968352 -0.0017449929298968211 -0.001744992929896838 Al
-0.0017449929298968352 0.4982556074136683 0.4982556074136684 Al
0.4982556074136684 -0.0017449929298968226 0.4982556074136684 Al
0.4982556074136684 0.4982556074136683 -0.001744992929896838 Al
 []


/usr/local/lib/python3.11/dist-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['rotations']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


vacs= [OrderedDict([ ( 'atoms',
                OrderedDict([ ( 'lattice_mat',
                                [ [8.16331, 0.0, 0.0],
                                  [0.0, 8.16331, 0.0],
                                  [0.0, 0.0, 8.16331]]),
                              ( 'coords',
                                [ [ -0.007122450392313417,
                                    -0.0071224503923133595,
                                    -0.007122450392313428],
                                  [ -0.007122450392313417,
                                    -0.0071224503923133595,
                                    4.074532549607686],
                                  [ -0.007122450392313417,
                                    4.074532549607686,
                                    -0.007122450392313428],
                                  [ -0.007122450392313417,
                                    4.074532549607686,
                                    4.074532549607686],
              

In [13]:
from jarvis.io.lammps.outputs import parse_material_calculation_folder
fold = '/content/lammps/src/Test/Test/Test'
x = parse_material_calculation_folder(fold)

/content/lammps/src/Test/Test/Test
Found 3 folders


In [14]:
x

{'jid': 'x',
 'source_folder': '/content/lammps/src/Test/Test/Test',
 'bulk_energy_per_atom': '',
 'chem_pot': {},
 'surface_info': [],
 'vacancy_info': []}